In [1]:
import pandas as pd
import numpy as np

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [93]:
def get_new_id(teams):
    id_list = sorted(teams['team_id'].unique())

    if len(id_list) == 0:
        return 1
    prev = id_list[0]
    for id in id_list[1:]:
        if prev + 1 != id:
            return prev + 1
        prev = id
    return prev + 1

In [816]:
leagues = get_leagues(where='L.Region="Europe" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,Baltic Masters,BM,Europe,Secondary,Yes
1,Belgian League,BL,Europe,Secondary,Yes
2,Dutch League,DL,Europe,Secondary,Yes
3,Elite Series: League of Legends,ESLOL,Europe,Secondary,Yes
4,Esports Balkan League,EBL,Europe,Secondary,Yes
5,Europe Challenger Series,EU CS,Europe,Secondary,Yes
6,Europe League Championship Series,EU LCS,Europe,Primary,Yes
7,European Masters,EM,Europe,Secondary,Yes
8,Greek Legends League,GLL,Europe,Secondary,Yes
9,Hitpoint Challengers,HC,Europe,Secondary,Yes


In [821]:
tournaments = get_tournaments(where='L.League_Short="EU LCS" or L.League_Short="LoL EMEA Championship"')
tournaments = tournaments.sort_values(by=['Year', 'DateStart', 'Date']).reset_index(drop=True)
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,EU LCS 2013 Spring,EU LCS/Season 3/Spring Season,2013-02-09,2013-04-14,Europe League Championship Series,Europe,Offline,EU LCS 2013 Spring,Spring,1,Primary,0,0,1,2013,1,1
1,EU LCS 2013 Spring Playoffs,EU LCS/Season 3/Spring Playoffs,2013-04-26,2013-04-28,Europe League Championship Series,Europe,Offline,EU LCS 2013 Spring Playoffs,Spring,None,Primary,0,1,1,2013,1,1
2,EU LCS 2013 Summer Promotion,EU LCS/Season 3/Summer Promotion,2013-05-03,2013-05-06,Europe League Championship Series,Europe,Offline,EU LCS 2013 Summer Promotion,Summer,None,Primary,1,0,1,2013,1,1
3,EU LCS 2013 Summer,EU LCS/Season 3/Summer Season,2013-06-15,2013-08-17,Europe League Championship Series,Europe,Offline,EU LCS 2013 Summer,Summer,2,Primary,0,0,1,2013,1,1
4,EU LCS 2013 Summer Playoffs,EU LCS/Season 3/Summer Playoffs,2013-08-23,2013-08-25,Europe League Championship Series,Europe,Offline,EU LCS 2013 Summer Playoffs,Summer,None,Primary,0,1,1,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,EU LCS 2013 Lille Qualifier,EU LCS/Season 3/Summer Promotion/Qualifiers/Lille,2013-03-23,2013-03-24,Europe League Championship Series,Europe,Offline,None,None,None,None,1,None,None,None,1,1
63,EU LCS 2013 Summer Go4LoL Qualifiers,EU LCS/Season 3/Summer Promotion/Qualifiers/Go...,2013-03-31,2013-04-11,Europe League Championship Series,Europe,Online,None,None,None,None,1,None,None,None,1,1
64,EU LCS 2013 Summer Ranked 5s Qualifiers,EU LCS/Season 3/Summer Promotion/Qualifiers/Ra...,2013-04-12,2013-04-14,Europe League Championship Series,Europe,Online,None,None,None,None,1,None,None,None,1,1
65,EU LCS 2014 Tenerife Online Qualifiers,EU LCS/2014 Season/Spring Promotion/Qualifiers...,2013-07-04,2013-07-11,Europe League Championship Series,Europe,Online,None,None,None,None,1,None,None,None,1,1


In [923]:
for i in range(len(tournaments)):
    scoreboard_games = get_scoreboard_games(
        where=f'T.OverviewPage="{tournaments["OverviewPage"][i]}"'
    )
    if scoreboard_games is not None:
        scoreboard_games = scoreboard_games.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        team_names = scoreboard_games[['Team1', 'Team2']].unstack().unique()
        teams = pd.read_csv('teams_id.csv')
        names = []
        for name in team_names:
            if name not in teams['team'].values:
                names.append(name)
        if len(names) > 0:
            names = sorted(names)
            break
print(scoreboard_games['OverviewPage'].unique())
print(names)

['EU LCS/2015 Season/Spring Season']
['Elements']


In [809]:
teams.loc[teams['team'].str.contains('Suning')]

,team,team_id
126,Suning,102


In [924]:
teams = pd.concat(
    [
        teams,
        pd.Series({'team': names.pop(0), 'team_id': get_new_id(teams)}).to_frame().T
        # pd.Series({'team': names.pop(0), 'team_id': 102}).to_frame().T
    ],
    ignore_index=True
)
print(names)
teams

[]


,team,team_id
0,2144 Danmu Gaming,91
1,AD Gaming,85
2,APK Prince,66
3,AT Gaming,126
4,Afreeca Freecs,56
...,...,...
219,headshotBG,152
220,mousesports,123
221,n!faculty,168
222,team propertybb2,139


In [925]:
names

[]

In [926]:
teams = teams.sort_values(by='team').reset_index(drop=True)
teams

,team,team_id
0,2144 Danmu Gaming,91
1,AD Gaming,85
2,APK Prince,66
3,AT Gaming,126
4,Afreeca Freecs,56
...,...,...
219,ex-GamingGear.EU,138
220,headshotBG,152
221,mousesports,123
222,n!faculty,168


In [927]:
teams.to_csv('teams_id.csv', index=False)